In [8]:
#manipulate dataframes in python
import pandas as pd
import sys

#make API calls with python
import requests

# sys.path.insert(0, '/path/to/application/app/folder') used to import local python file, in this case local keys
sys.path.insert(0, '../src/')
import localvars

#allows us to store results of API call cleanly
import json

# List of Years
strYears =["2010","2013", "2014", "2015", "2016", "2017"]

intState = 47
intCounty = 125

tables = pd.read_csv("../src/data/tables.csv").to_dict(orient="row")
tablesdf = pd.DataFrame(tables)
tablesdf2 = tablesdf.set_index("table", drop = False)
for index, row in tablesdf2.iterrows():
    tablename= row['table']
    #print("value: " + tablesdf2.loc[tablename,'file'])
    # Using for loop 
    metric = pd.DataFrame() #creates a new dataframe that's empty
    for i in strYears: 

        #construct the API call we will use
        if tablename[:1]=="B":
            baseAPI = "https://api.census.gov/data/%s/acs/acs5?get=%s&for=tract:*&in=state:%i%%20county:%i&key=%s" 
        elif tablename[:1]=="D":
            baseAPI = "https://api.census.gov/data/%s/acs/acs5/profile?get=%s&for=tract:*&in=state:%i%%20county:%i&key=%s" 
        elif tablename[:1]=="S":
            baseAPI = "https://api.census.gov/data/%s/acs/acs5/subject?get=%s&for=tract:*&in=state:%i%%20county:%i&key=%s"

        calledAPI = (baseAPI % (i, tablename, intState, intCounty, localvars.apiKey))

        #call the API and collect the response
        response = requests.get(calledAPI)

        #load the response into a JSON, ignoring the first element which is just field labels
        try:
            formattedResponse = json.loads(response.text)[1:]

            #flip the order of the response from [population, zipcode] -> [zipcode, population]
            formattedResponse = [item[::-1] for item in formattedResponse]

            #store the response in a dataframe
            strColYear = "y_" + i

            metrictemp = pd.DataFrame(columns=['id', '2', '3', strColYear], data=formattedResponse)
            del metrictemp['2']
            del metrictemp['3']
            if i == "2010":
                metric=metrictemp
            else: 
                metric = metric.merge(metrictemp, on='id', how='outer')
        except:
            print("Unexpected error:", sys.exc_info()[0])
            raise

    #save that dataframe to a CSV spreadsheet
    strFileName=row['file']
    strFile = "../src/data/ProcessedData/" + strFileName + ".csv"
    strJson ="../src/data/json/m" + strFileName[1:]+ ".json"
    metric.to_csv(strFile, index=False)
    print(calledAPI)
    print("Output: " + strFile)




https://api.census.gov/data/2017/acs/acs5?get=B01001_001E&for=tract:*&in=state:47%20county:125&key=c565c84c8194e4fac85204a34580b7ced55fa09c
Output: ../src/data/ProcessedData/r1.csv
https://api.census.gov/data/2017/acs/acs5?get=B01001_002E&for=tract:*&in=state:47%20county:125&key=c565c84c8194e4fac85204a34580b7ced55fa09c
Output: ../src/data/ProcessedData/r2.csv
https://api.census.gov/data/2017/acs/acs5?get=B01001_026E&for=tract:*&in=state:47%20county:125&key=c565c84c8194e4fac85204a34580b7ced55fa09c
Output: ../src/data/ProcessedData/r3.csv
https://api.census.gov/data/2017/acs/acs5?get=B01001A_001E&for=tract:*&in=state:47%20county:125&key=c565c84c8194e4fac85204a34580b7ced55fa09c
Output: ../src/data/ProcessedData/r4.csv
https://api.census.gov/data/2017/acs/acs5?get=B01001B_001E&for=tract:*&in=state:47%20county:125&key=c565c84c8194e4fac85204a34580b7ced55fa09c
Output: ../src/data/ProcessedData/r5.csv
https://api.census.gov/data/2017/acs/acs5?get=B01001C_001E&for=tract:*&in=state:47%20county:1

In [3]:
import pandas as pd
import csv  
import json 

df = pd.read_csv("../src/data/ProcessedData/r1.csv", index_col='id', names=['id', 'y_2010', 'y_2013', 'y_2014', 'y_2015', 'y_2016', 'y_2017'])
df["y_2010"] = pd.to_numeric(df['y_2010'], errors='coerce').fillna(0)
df["y_2013"] = pd.to_numeric(df['y_2013'], errors='coerce').fillna(0)
df["y_2014"] = pd.to_numeric(df['y_2014'], errors='coerce').fillna(0)
df["y_2015"] = pd.to_numeric(df['y_2015'], errors='coerce').fillna(0)
df["y_2016"] = pd.to_numeric(df['y_2016'], errors='coerce').fillna(0)
df["y_2017"] = pd.to_numeric(df['y_2017'], errors='coerce').fillna(0)

export = df.to_json('../src/data/json/m1.json', orient='index')


In [4]:
import csv  
import json  
  
csvFilePath = '../src/data/ProcessedData/r1.csv'
jsonFilePath = '../src/data/json/m1.json'

data = {}

with open(csvFilePath) as csvFile:
    csvReader=csv.DictReader(csvFile)
    for rows in csvReader:
        id=rows['id']
        #del rows['id']
        data[id]=rows       
        print(data[id])
    
with open(jsonFilePath,'w') as jsonFile:
    jsonFile.write(json.dumps(data,indent=4))


OrderedDict([('id', '100100'), ('y_2010', '1006'), ('y_2013', '1173'), ('y_2014', '1274'), ('y_2015', '1233'), ('y_2016', '1232'), ('y_2017', '1203')])
OrderedDict([('id', '100200'), ('y_2010', '1842'), ('y_2013', '1776'), ('y_2014', '1805'), ('y_2015', '1902'), ('y_2016', '1606'), ('y_2017', '1464')])
OrderedDict([('id', '100300'), ('y_2010', '5021'), ('y_2013', '5567'), ('y_2014', '5727'), ('y_2015', '5436'), ('y_2016', '5890'), ('y_2017', '5398')])
OrderedDict([('id', '100400'), ('y_2010', '2479'), ('y_2013', '3236'), ('y_2014', '3406'), ('y_2015', '3402'), ('y_2016', '3390'), ('y_2017', '3239')])
OrderedDict([('id', '100500'), ('y_2010', '4880'), ('y_2013', '4598'), ('y_2014', '5233'), ('y_2015', '5215'), ('y_2016', '4956'), ('y_2017', '4813')])
OrderedDict([('id', '100601'), ('y_2010', '2278'), ('y_2013', '2278'), ('y_2014', '2125'), ('y_2015', '2212'), ('y_2016', '2301'), ('y_2017', '2201')])
OrderedDict([('id', '100602'), ('y_2010', '2948'), ('y_2013', '2855'), ('y_2014', '2648'